In [216]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [217]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [218]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [219]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [220]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [221]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [222]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [223]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [224]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [225]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [226]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [227]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [228]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Nov-20,"25,334","24,482",852,3.4,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Nov-20,"92,160","88,745","3,415",3.7,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Nov-20,"9,733","9,190",543,5.6,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Nov-20,"8,627","8,251",376,4.4,01007
38631,CN0100900000000,1,9,"Blount County, AL",Nov-20,"24,849","24,210",639,2.6,01009


In [229]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [230]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Nov-20,"12,447","10,911",1536,12.3,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Nov-20,"2,445","2,166",279,11.4,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Nov-20,"6,613","5,859",754,11.4,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Nov-20,"7,907","7,027",880,11.1,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Nov-20,"9,202","8,089",1113,12.1,72153


In [231]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [232]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [233]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [234]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [235]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [236]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [237]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [238]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [239]:
covid.tail(50)

,date,county,state,fips,cases,deaths
1021023,2021-02-11,Ozaukee,Wisconsin,55089.0,8379,79.0
1021024,2021-02-11,Pepin,Wisconsin,55091.0,809,7.0
1021025,2021-02-11,Pierce,Wisconsin,55093.0,4263,39.0
1021026,2021-02-11,Polk,Wisconsin,55095.0,3795,44.0
1021027,2021-02-11,Portage,Wisconsin,55097.0,6821,65.0
1021028,2021-02-11,Price,Wisconsin,55099.0,1220,7.0
1021029,2021-02-11,Racine,Wisconsin,55101.0,22619,343.0
1021030,2021-02-11,Richland,Wisconsin,55103.0,1288,13.0
1021031,2021-02-11,Rock,Wisconsin,55105.0,15609,162.0
1021032,2021-02-11,Rusk,Wisconsin,55107.0,1280,17.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [240]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
342,02/05/2021,3908,1209,368,68,11,3184,4169,328,71,79,904,247,103,11,0,655,862,89,13,13,1172,327,113,26,1,956,1233,101,24,25,512,226,57,9,0,457,628,48,9,10,1094,325,79,18,2,921,1182,76,20,22,223,84,16,4,0,193,251,14,4,4,11000
343,02/06/2021,2881,952,302,65,12,3203,4191,336,70,78,678,197,91,9,1,674,883,93,12,13,821,287,83,24,0,955,1234,104,24,25,367,171,54,7,1,461,638,49,9,10,846,234,62,19,1,916,1174,76,20,21,168,60,12,6,0,194,253,14,5,5,11000
344,02/07/2021,1527,720,268,60,15,3027,3962,336,69,78,296,148,76,17,0,639,838,95,14,14,522,216,83,14,1,910,1180,104,22,24,187,120,39,12,0,443,608,50,10,10,431,183,59,16,0,856,1098,74,19,21,90,53,11,1,0,177,235,13,4,4,11000
345,02/08/2021,3858,1142,184,59,17,3444,4454,325,69,80,834,257,77,16,1,735,952,94,14,15,1193,329,33,16,0,1035,1323,98,22,23,540,160,33,11,1,495,668,50,10,11,1063,315,35,12,1,975,1239,71,18,20,228,81,6,4,0,203,269,12,4,4,11000
346,02/09/2021,2548,1057,26,40,24,3443,4518,277,64,77,641,209,18,6,1,767,996,82,13,14,749,322,7,15,0,1023,1332,82,19,20,343,185,1,7,1,482,667,43,10,11,645,273,0,10,1,963,1242,59,17,18,169,66,0,2,0,207,278,10,4,4,11000


In [241]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [242]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [243]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [244]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [245]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [246]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [247]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
342,02/05/2021,904,11,New York,Bronx,36005.0
343,02/06/2021,678,9,New York,Bronx,36005.0
344,02/07/2021,296,17,New York,Bronx,36005.0
345,02/08/2021,834,16,New York,Bronx,36005.0
346,02/09/2021,641,6,New York,Bronx,36005.0


In [248]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [249]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [250]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [251]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
342,02/05/2021,1172,26,Kings,New York,36047.0
343,02/06/2021,821,24,Kings,New York,36047.0
344,02/07/2021,522,14,Kings,New York,36047.0
345,02/08/2021,1193,16,Kings,New York,36047.0
346,02/09/2021,749,15,Kings,New York,36047.0


In [252]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [253]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [254]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [255]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [256]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,11051,264


In [257]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [258]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [259]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
333,01/27/2021,1343,21,New York,Queens,36081.0,2021-01-27
334,01/28/2021,1289,27,New York,Queens,36081.0,2021-01-28
335,01/29/2021,1036,27,New York,Queens,36081.0,2021-01-29
336,01/30/2021,879,22,New York,Queens,36081.0,2021-01-30
337,01/31/2021,857,20,New York,Queens,36081.0,2021-01-31
338,02/01/2021,227,20,New York,Queens,36081.0,2021-02-01
339,02/02/2021,732,20,New York,Queens,36081.0,2021-02-02
340,02/03/2021,1326,18,New York,Queens,36081.0,2021-02-03
341,02/04/2021,1333,24,New York,Queens,36081.0,2021-02-04
342,02/05/2021,1094,18,New York,Queens,36081.0,2021-02-05


In [260]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
297,12/22/2020,Richmond,New York,36085.0,362,10.0,2020-12-22
298,12/23/2020,Richmond,New York,36085.0,396,5.0,2020-12-23
299,12/24/2020,Richmond,New York,36085.0,239,4.0,2020-12-24
300,12/25/2020,Richmond,New York,36085.0,96,7.0,2020-12-25
301,12/26/2020,Richmond,New York,36085.0,392,1.0,2020-12-26
302,12/27/2020,Richmond,New York,36085.0,311,9.0,2020-12-27
303,12/28/2020,Richmond,New York,36085.0,561,7.0,2020-12-28
304,12/29/2020,Richmond,New York,36085.0,584,9.0,2020-12-29
305,12/30/2020,Richmond,New York,36085.0,463,4.0,2020-12-30
306,12/31/2020,Richmond,New York,36085.0,385,3.0,2020-12-31


In [261]:
print (data_date_dt)

2021-02-12 00:00:00


In [262]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1017828,1001.0,81.0
1017829,1003.0,240.0


In [263]:
Kings_death.head(2)

,fips,deaths
0,36047.0,6944


In [264]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1020828,51690.0,40.0
1020829,51115.0,7.0
1020830,51117.0,46.0
1020831,51119.0,16.0
1020832,51121.0,70.0
1020833,51125.0,7.0
1020834,51127.0,7.0
1020835,51700.0,113.0
1020836,51710.0,149.0
1020837,51131.0,33.0


In [265]:
covid_death.shape

(3250, 2)

In [266]:
print (County_covid_death)

{1001.0: 81.0, 1003.0: 240.0, 1005.0: 46.0, 1007.0: 54.0, 1009.0: 116.0, 1011.0: 32.0, 1013.0: 64.0, 1015.0: 257.0, 1017.0: 92.0, 1019.0: 37.0, 1021.0: 87.0, 1023.0: 23.0, 1025.0: 43.0, 1027.0: 53.0, 1029.0: 33.0, 1031.0: 90.0, 1033.0: 112.0, 1035.0: 23.0, 1037.0: 22.0, 1039.0: 96.0, 1041.0: 51.0, 1043.0: 169.0, 1045.0: 103.0, 1047.0: 126.0, 1049.0: 162.0, 1051.0: 165.0, 1053.0: 56.0, 1055.0: 288.0, 1057.0: 49.0, 1059.0: 74.0, 1061.0: 62.0, 1063.0: 32.0, 1065.0: 57.0, 1067.0: 39.0, 1069.0: 244.0, 1071.0: 86.0, 1073.0: 1265.0, 1075.0: 32.0, 1077.0: 196.0, 1079.0: 83.0, 1081.0: 137.0, 1083.0: 125.0, 1085.0: 42.0, 1087.0: 40.0, 1089.0: 400.0, 1091.0: 44.0, 1093.0: 87.0, 1095.0: 191.0, 1097.0: 667.0, 1099.0: 32.0, 1101.0: 449.0, 1103.0: 230.0, 1105.0: 25.0, 1107.0: 47.0, 1109.0: 67.0, 1111.0: 38.0, 1113.0: 29.0, 1117.0: 192.0, 1115.0: 206.0, 1119.0: 29.0, 1121.0: 143.0, 1123.0: 129.0, 1125.0: 383.0, 1127.0: 234.0, 1129.0: 31.0, 1131.0: 23.0, 1133.0: 61.0, 2013.0: 0.0, 2016.0: 0.0, 2020.0: 

In [267]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [268]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [269]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
975943,2021-01-29,New Haven,Connecticut,9009.0,63085,1786.0,2021-01-29,-14 days,NaN
979188,2021-01-30,New Haven,Connecticut,9009.0,63085,1786.0,2021-01-30,-13 days,0.0
982434,2021-01-31,New Haven,Connecticut,9009.0,63085,1786.0,2021-01-31,-12 days,0.0
985681,2021-02-01,New Haven,Connecticut,9009.0,63937,1803.0,2021-02-01,-11 days,852.0
988928,2021-02-02,New Haven,Connecticut,9009.0,64733,1805.0,2021-02-02,-10 days,796.0
992174,2021-02-03,New Haven,Connecticut,9009.0,64889,1810.0,2021-02-03,-9 days,156.0
995420,2021-02-04,New Haven,Connecticut,9009.0,65075,1815.0,2021-02-04,-8 days,186.0
998666,2021-02-05,New Haven,Connecticut,9009.0,65488,1820.0,2021-02-05,-7 days,413.0
1001912,2021-02-06,New Haven,Connecticut,9009.0,65488,1820.0,2021-02-06,-6 days,0.0
1005158,2021-02-07,New Haven,Connecticut,9009.0,65488,1820.0,2021-02-07,-5 days,0.0


In [270]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [271]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,4295.0


In [272]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
297,12/22/2020,Richmond,New York,36085.0,362,10.0,2020-12-22
298,12/23/2020,Richmond,New York,36085.0,396,5.0,2020-12-23
299,12/24/2020,Richmond,New York,36085.0,239,4.0,2020-12-24
300,12/25/2020,Richmond,New York,36085.0,96,7.0,2020-12-25
301,12/26/2020,Richmond,New York,36085.0,392,1.0,2020-12-26
302,12/27/2020,Richmond,New York,36085.0,311,9.0,2020-12-27
303,12/28/2020,Richmond,New York,36085.0,561,7.0,2020-12-28
304,12/29/2020,Richmond,New York,36085.0,584,9.0,2020-12-29
305,12/30/2020,Richmond,New York,36085.0,463,4.0,2020-12-30
306,12/31/2020,Richmond,New York,36085.0,385,3.0,2020-12-31


In [273]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [274]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [275]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [276]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [277]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,16.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [278]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-02-13 02:38:08.834483
2


In [279]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [280]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [281]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [282]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [283]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 81.0
1003.0 240.0
1005.0 46.0
1007.0 54.0
1009.0 116.0
1011.0 32.0
1013.0 64.0
1015.0 257.0
1017.0 92.0
1019.0 37.0
1021.0 87.0
1023.0 23.0
1025.0 43.0
1027.0 53.0
1029.0 33.0
1031.0 90.0
1033.0 112.0
1035.0 23.0
1037.0 22.0
1039.0 96.0
1041.0 51.0
1043.0 169.0
1045.0 103.0
1047.0 126.0
1049.0 162.0
1051.0 165.0
1053.0 56.0
1055.0 288.0
1057.0 49.0
1059.0 74.0
1061.0 62.0
1063.0 32.0
1065.0 57.0
1067.0 39.0
1069.0 244.0
1071.0 86.0
1073.0 1265.0
1075.0 32.0
1077.0 196.0
1079.0 83.0
1081.0 137.0
1083.0 125.0
1085.0 42.0
1087.0 40.0
1089.0 400.0
1091.0 44.0
1093.0 87.0
1095.0 191.0
1097.0 667.0
1099.0 32.0
1101.0 449.0
1103.0 230.0
1105.0 25.0
1107.0 47.0
1109.0 67.0
1111.0 38.0
1113.0 29.0
1117.0 192.0
1115.0 206.0
1119.0 29.0
1121.0 143.0
1123.0 129.0
1125.0 383.0
1127.0 234.0
1129.0 31.0
1131.0 23.0
1133.0 61.0
2013.0 0.0
2016.0 0.0
2020.0 149.0
2050.0 17.0
2997.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 26.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 2.0
2150.0 5.0
2158.0 3.0
2170.0 29.

22087.0 46.0
22089.0 88.0
22091.0 8.0
22093.0 48.0
22095.0 130.0
22097.0 234.0
22099.0 108.0
22101.0 117.0
22103.0 463.0
22105.0 254.0
22107.0 6.0
22109.0 189.0
22111.0 74.0
nan 0.0
22113.0 103.0
22115.0 92.0
22117.0 116.0
22119.0 91.0
22121.0 51.0
22123.0 38.0
22125.0 30.0
22127.0 38.0
23001.0 54.0
23003.0 43.0
23005.0 169.0
23007.0 9.0
23009.0 31.0
23011.0 47.0
23013.0 5.0
23015.0 1.0
23017.0 41.0
23019.0 73.0
23021.0 1.0
23023.0 1.0
23025.0 26.0
23027.0 18.0
23029.0 13.0
23031.0 109.0
24001.0 193.0
24003.0 501.0
24005.0 1205.0
24510.0 848.0
24009.0 63.0
24011.0 18.0
24013.0 206.0
24015.0 113.0
24017.0 151.0
24019.0 39.0
24021.0 257.0
24023.0 58.0
24025.0 221.0
24027.0 214.0
24029.0 39.0
24031.0 1363.0
24033.0 1280.0
24035.0 37.0
24039.0 28.0
24037.0 108.0
24041.0 29.0
nan 33.0
24043.0 240.0
24045.0 136.0
24047.0 87.0
25001.0 368.0
25003.0 239.0
25005.0 1412.0
25007.0 0.0
25009.0 2058.0
25011.0 99.0
25013.0 1259.0
25015.0 244.0
25017.0 3265.0
25019.0 2.0
25021.0 1545.0
25023.0 1221.0

48013.0 126.0
48015.0 26.0
48017.0 16.0
48019.0 22.0
48021.0 69.0
48023.0 11.0
48025.0 60.0
48027.0 304.0
48029.0 2611.0
48031.0 15.0
48033.0 2.0
48035.0 28.0
48037.0 167.0
48039.0 342.0
48041.0 200.0
48043.0 10.0
48045.0 4.0
48047.0 32.0
48049.0 117.0
48051.0 30.0
48053.0 51.0
48055.0 70.0
48057.0 15.0
48059.0 37.0
48061.0 1392.0
48063.0 40.0
48065.0 12.0
48067.0 73.0
48069.0 27.0
48071.0 34.0
48073.0 116.0
48075.0 15.0
48077.0 16.0
48079.0 13.0
48081.0 11.0
48083.0 27.0
48085.0 653.0
48087.0 9.0
48089.0 24.0
48091.0 269.0
48093.0 46.0
48095.0 7.0
48097.0 57.0
48099.0 65.0
48101.0 7.0
48103.0 11.0
48105.0 15.0
48107.0 25.0
48109.0 7.0
48111.0 22.0
48113.0 3100.0
48115.0 68.0
48123.0 55.0
48117.0 63.0
48119.0 8.0
48121.0 580.0
48125.0 8.0
48127.0 19.0
48129.0 12.0
48131.0 36.0
48133.0 37.0
48135.0 307.0
48137.0 4.0
48141.0 2220.0
48139.0 263.0
48143.0 62.0
48145.0 28.0
48147.0 91.0
48149.0 50.0
48151.0 13.0
48153.0 29.0
48155.0 7.0
48157.0 517.0
48159.0 21.0
48161.0 38.0
48163.0 38.0
4

In [284]:
K_c.head()

,fips,cases,deaths
0,36047.0,11051,264


In [285]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [286]:
print (K)

{36047.0: 11051}


In [287]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 149.0
1001 374.0
1009 300.0
1013 86.0
1015 621.0
1017 131.0
1031 268.0
1033 196.0
1039 272.0
1043 269.0
1045 206.0
1051 499.0
1055 465.0
1067 112.0
1069 452.0
1071 201.0
1077 298.0
1079 127.0
1083 479.0
1089 1886.0
1095 343.0
1097 1808.0
1103 393.0
1111 54.0
1113 211.0
1115 433.0
1117 1085.0
1121 451.0
2261 19.0
4021 2387.0
5009 221.0
5011 75.0
5033 468.0
5037 56.0
5045 867.0
5047 76.0
5051 870.0
5053 149.0
5063 173.0
5083 146.0
5085 629.0
5087 55.0
5115 337.0
5117 76.0
5121 82.0
5125 817.0
5131 695.0
5145 542.0
6007 557.0
6017 484.0
6023 255.0
6027 102.0
6061 776.0
6079 1284.0
6089 430.0
6093 118.0
8019 11.0
8039 78.0
8047 13.0
8051 58.0
8057 3.0
8065 99.0
8067 123.0
8071 44.0
8075 65.0
8077 854.0
8083 65.0
8085 147.0
8093 37.0
8099 37.0
8101 160.0
8103 33.0
8119 45.0
8121 4.0
9005 686.0
9007 635.0
9011 1399.0
9015 634.0
10001 918.0
12019 910.0
12033 2103.0
12089 500.0
12101 2017.0
12109 1258.0
12113 1059.0
13015 916.0
13039 191.0
13045 564.0
13047 445.0
13057 1519.0
13073 984.0

37057 1125.0
37059 362.0
37069 403.0
37071 1357.0
37077 317.0
37087 304.0
37097 1516.0
37101 1374.0
37109 608.0
37115 85.0
37121 83.0
37123 171.0
37125 537.0
37127 1053.0
37139 252.0
37141 314.0
37145 305.0
37151 1062.0
37157 367.0
37159 1241.0
37161 328.0
37167 429.0
37169 346.0
37171 672.0
37197 210.0
38001 7.0
38003 3.0
38007 0.0
38009 18.0
38011 18.0
38021 6.0
38025 1.0
38027 14.0
38031 8.0
38033 2.0
38041 5.0
38043 0.0
38049 13.0
38055 19.0
38057 7.0
38059 54.0
38065 2.0
38067 15.0
38069 2.0
38073 8.0
38075 1.0
38081 6.0
38091 2.0
38095 1.0
38097 10.0
38099 13.0
39003 360.0
39005 121.0
39007 567.0
39011 145.0
39013 354.0
39015 286.0
39017 1839.0
39019 106.0
39021 131.0
39023 541.0
39025 949.0
39027 231.0
39029 278.0
39033 127.0
39037 138.0
39039 134.0
39043 268.0
39045 682.0
39051 183.0
39053 99.0
39055 385.0
39057 666.0
39059 115.0
39063 197.0
39065 119.0
39067 54.0
39069 100.0
39071 246.0
39073 103.0
39077 169.0
39081 191.0
39083 206.0
39085 926.0
39089 690.0
39091 178.0
39093 1

44003 1083.0
45017 60.0
45019 3192.0
45021 522.0
45037 212.0
45039 152.0
45041 920.0
45043 375.0
45047 588.0
45051 2848.0
45059 573.0
45071 286.0
46003 1.0
46009 4.0
46013 123.0
46021 4.0
46023 17.0
46033 10.0
46035 28.0
46037 27.0
46047 11.0
46057 11.0
46061 15.0
46065 35.0
46069 1.0
46073 2.0
46075 0.0
46087 12.0
46091 5.0
46099 482.0
46103 224.0
46127 82.0
46135 33.0
47005 61.0
47017 124.0
47023 138.0
47045 183.0
47059 308.0
47073 266.0
47085 67.0
47093 2275.0
47105 227.0
47109 83.0
47123 254.0
47125 1027.0
47153 66.0
47169 50.0
47187 1308.0
48007 122.0
48035 502.0
48037 481.0
48045 12.0
48071 332.0
48093 91.0
48121 6072.0
48125 14.0
48129 17.0
48133 115.0
48147 284.0
48161 828.0
48167 2610.0
48185 1329.0
48199 263.0
48209 1234.0
48213 410.0
48217 1690.0
48231 307.0
48237 22.0
48239 97.0
48257 1173.0
48265 451.0
48269 1.0
48285 155.0
48287 577.0
48289 582.0
48293 1015.0
48295 24.0
48303 871.0
48331 227.0
48349 163.0
48351 31.0
48353 20.0
48365 94.0
48379 51.0
48393 2.0
48397 808.0
4

48443 1.0
48449 222.0
48457 50.0
48471 589.0
48473 289.0
48495 24.0
49039 167.0
51029 101.0
51081 93.0
51153 2671.0
51590 403.0
51620 70.0
51660 243.0
51683 235.0
51685 34.0
51690 87.0
51700 911.0
51740 605.0
51760 1135.0
53025 355.0
54013 6.0
54109 127.0
1065 86.0
2013 265.0
2185 11.0
4009 154.0
4012 122.0
5035 253.0
5095 80.0
5099 38.0
6039 624.0
6071 7870.0
6075 1762.0
6077 2808.0
6099 3225.0
12013 49.0
12077 28.0
12079 80.0
12086 21382.0
12093 188.0
12123 110.0
12125 45.0
13019 96.0
13063 2395.0
13095 288.0
13099 57.0
13109 43.0
13215 769.0
13261 168.0
13267 141.0
13301 33.0
13303 110.0
16065 212.0
20057 51.0
21013 143.0
21045 91.0
21065 50.0
21075 21.0
21109 69.0
21121 201.0
21129 38.0
21203 73.0
24510 1506.0
26085 10.0
28013 44.0
29223 26.0
30035 57.0
31043 27.0
31047 53.0
34011 821.0
34017 3872.0
34031 3077.0
34039 2736.0
35005 283.0
35019 9.0
35023 16.0
35033 5.0
35045 397.0
35051 9.0
35053 78.0
36047 1741.0
36081 1738.0
36087 2073.0
37083 604.0
38005 1.0
40001 213.0
40139 76.0

In [288]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 2228
36061 5155
36047 11051 11051
36081 10469
36005 7738


In [289]:
print (covid_cases_county_ny_times)

{1001.0: 374.0, 1003.0: 1038.0, 1005.0: 99.0, 1007.0: 93.0, 1009.0: 300.0, 1011.0: 64.0, 1013.0: 86.0, 1015.0: 621.0, 1017.0: 131.0, 1019.0: 50.0, 1021.0: 141.0, 1023.0: 12.0, 1025.0: 185.0, 1027.0: 63.0, 1029.0: 69.0, 1031.0: 268.0, 1033.0: 196.0, 1035.0: 44.0, 1037.0: 149.0, 1039.0: 272.0, 1041.0: 91.0, 1043.0: 269.0, 1045.0: 206.0, 1047.0: 78.0, 1049.0: 243.0, 1051.0: 499.0, 1053.0: 193.0, 1055.0: 465.0, 1057.0: 87.0, 1059.0: 105.0, 1061.0: 105.0, 1063.0: 47.0, 1065.0: 86.0, 1067.0: 112.0, 1069.0: 452.0, 1071.0: 201.0, 1073.0: 2794.0, 1075.0: 40.0, 1077.0: 298.0, 1079.0: 127.0, 1081.0: 812.0, 1083.0: 479.0, 1085.0: 78.0, 1087.0: 87.0, 1089.0: 1886.0, 1091.0: 76.0, 1093.0: 94.0, 1095.0: 343.0, 1097.0: 1808.0, 1099.0: 80.0, 1101.0: 1091.0, 1103.0: 393.0, 1105.0: 60.0, 1107.0: 108.0, 1109.0: 130.0, 1111.0: 54.0, 1113.0: 211.0, 1115.0: 433.0, 1117.0: 1085.0, 1119.0: 35.0, 1121.0: 451.0, 1123.0: 157.0, 1125.0: 1050.0, 1127.0: 194.0, 1129.0: 54.0, 1131.0: 58.0, 1133.0: 49.0, 2013.0: 265.0

In [290]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [291]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [292]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [332]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [294]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [295]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [296]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [297]:
#ACI
ACI_State = total_state(ACI_total)

In [298]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [299]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [300]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [301]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [335]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [303]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [304]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 61.0 38921 156.72773053107576
1001 81.0 55869 144.98201149116684
1009 116.0 57826 200.60180541624874
1013 64.0 19448 329.0826820238585
1015 257.0 113605 226.2224373927204
1017 92.0 33254 276.6584471041078
1031 90.0 52342 171.94604715142717
1033 112.0 55241 202.74795894353832
1039 96.0 37049 259.116305433345
1043 169.0 83768 201.74768407983956
1045 103.0 49172 209.4688033840397
1051 165.0 81209 203.17945055351007
1055 288.0 102268 281.61301677944226
1067 39.0 17205 226.6782911944202
1069 244.0 105882 230.44521259515312
1071 86.0 51626 166.58272963235578
1077 196.0 92729 211.36861176115346
1079 83.0 32924 252.09573563358038
1083 125.0 98915 126.37112672496589
1089 400.0 372909 107.26477505235862
1095 191.0 96774 197.36706140078948
1097 667.0 413210 161.4191331284335
1103 230.0 119679 192.18075017338046
1111 38.0 22722 167.23879940146114
1113 29.0 57961 50.03364331188213
1115 206.0 89512 230.13674144248816
1117 192.0 217702 88.19395320208358
1121 143.0 79978 178.79916977168722
2261 

29059 22.0 16878 130.347197535253
29061 11.0 8278 132.88233872916163
29071 148.0 103967 142.3528619658161
29073 36.0 14706 244.79804161566705
29075 20.0 6571 304.36767615279257
29079 34.0 9850 345.1776649746193
29081 14.0 8352 167.62452107279694
29083 33.0 21824 151.20967741935485
29087 10.0 4403 227.11787417669774
29089 5.0 10001 49.99500049995
29091 42.0 40117 104.69377072064212
29097 109.0 121328 89.83911380720032
29099 195.0 225081 86.6354778946246
29103 2.0 3959 50.51780752715332
29105 64.0 35723 179.15628586624865
29107 48.0 32708 146.75308792955852
29109 67.0 38355 174.68387433189937
29113 30.0 59013 50.836256418077376
29115 11.0 11920 92.28187919463087
29121 12.0 15117 79.38082952966859
29125 7.0 8697 80.48752443371278
29127 39.0 28530 136.6982124079916
29129 2.0 3617 55.294442908487696
29131 48.0 25619 187.36094305008
29137 10.0 8644 115.68718186024988
29139 10.0 11551 86.57259111765215
29141 39.0 20627 189.07257478062732
29145 63.0 58236 108.18050690294663
29151 12.0 13615 88

39149 28.0 48590 57.62502572545791
39151 538.0 370606 145.1676443446679
39153 843.0 541013 155.81880657211565
39155 351.0 197974 177.2960085667815
39157 210.0 91987 228.29312837683585
39159 37.0 58988 62.72462195700821
39161 47.0 28275 166.22458001768345
39165 173.0 234602 73.7419118336587
39167 59.0 59911 98.47941112650432
39169 187.0 115710 161.6109238613776
39171 75.0 36692 204.40422980486207
39175 51.0 21772 234.24582031967665
40013 51.0 47995 106.26106886133972
40015 52.0 28762 180.79410333078368
40017 81.0 148306 54.616805793427105
40019 41.0 48111 85.21959635010704
40021 38.0 48657 78.09770433853299
40027 246.0 284014 86.61544853422718
40031 123.0 120749 101.8641976331067
40033 13.0 5666 229.43875750088245
40035 10.0 14142 70.71135624381276
40037 106.0 71522 148.20614636056038
40045 1.0 3859 25.91344908007256
40047 65.0 61056 106.45964360587001
40049 41.0 27711 147.95568546786475
40051 66.0 55834 118.20754379052191
40053 7.0 4333 161.55088852988692
40065 43.0 24530 175.295556461

2090 26.0 96849 26.845914774545946
2110 3.0 31974 9.382623381497467
2122 19.0 58708 32.363562035838385
2195 0.0 3266 0.0
2220 0.0 8493 0.0
2230 0.0 1183 0.0
2275 0.0 2502 0.0
4003 246.0 125922 195.3590317815791
4019 2023.0 1047279 193.16724578646188
4025 434.0 235099 184.60308210583625
5003 29.0 19657 147.530141934171
5005 96.0 41932 228.94209672803586
5007 372.0 279141 133.26598385762034
5015 40.0 28380 140.94432699083862
5019 36.0 22320 161.29032258064515
5023 65.0 24919 260.8451382479233
5025 23.0 7956 289.08999497234794
5029 31.0 20846 148.70958457257987
5031 174.0 110332 157.7058333031215
5043 34.0 18219 186.61836544266976
5055 72.0 45325 158.8527302813017
5059 68.0 33771 201.35619318350064
5061 24.0 13202 181.79063778215422
5071 30.0 26578 112.87531040710363
5073 7.0 6624 105.67632850241544
5075 41.0 16406 249.90857003535294
5081 40.0 12259 326.29088832694345
5091 32.0 43257 73.97646623667845
5103 48.0 23382 205.2861175263023
5105 7.0 10455 66.95361071257771
5109 21.0 10718 195.9

48485 322.0 132230 243.51508734780307
48493 55.0 51070 107.69532014881536
48497 104.0 69984 148.60539551897577
48499 108.0 45539 237.15935791299765
49003 40.0 56046 71.36994611569068
49005 29.0 128289 22.60521167052514
49007 11.0 20463 53.75555881346821
49011 126.0 355481 35.44493235925408
49013 9.0 19938 45.13993379476376
49015 8.0 10012 79.9041150619257
49019 2.0 9754 20.50440844781628
49021 26.0 54839 47.411513703750984
49027 10.0 13188 75.82650894752805
49029 11.0 12124 90.72913229957109
49033 0.0 2483 0.0
49041 7.0 21620 32.37742830712303
49043 8.0 42145 18.98208565666153
49045 27.0 72259 37.36558767765953
49047 17.0 35734 47.57373929590866
49051 18.0 34091 52.799859200375465
49057 154.0 260213 59.182285281673096
50001 8.0 36777 21.752725888462898
50007 82.0 163774 50.06899752097402
50009 0.0 6163 0.0
50021 9.0 58191 15.466309223075733
51001 33.0 32316 102.11659858893427
51003 39.0 109330 35.6718192627824
51036 10.0 6963 143.61625736033318
51037 11.0 11880 92.5925925925926
51087 3

48443 2.0 776 257.7319587628866
48449 73.0 32750 222.90076335877862
48457 31.0 21672 143.04171280915466
48471 111.0 72971 152.1152238560524
48473 36.0 55246 65.16308873040582
48495 19.0 8010 237.2034956304619
49039 19.0 30939 61.41116390316429
51029 20.0 17148 116.63167716351761
51081 18.0 11336 158.7861679604799
51153 327.0 470335 69.52491309385863
51590 89.0 40044 222.2555189291779
51620 23.0 7967 288.6908497552403
51660 68.0 53016 128.26316583672855
51683 33.0 41085 80.32128514056225
51685 8.0 17478 45.77182744021055
51690 40.0 12554 318.6235462800701
51700 113.0 179225 63.04923978239643
51740 107.0 94398 113.34985910718449
51760 144.0 230436 62.49023590064052
53025 108.0 97733 110.50515179110434
54013 0.0 7109 0.0
54109 32.0 20394 156.90889477297245
1065 57.0 14651 389.0519418469729
2013 0.0 3337 0.0
2185 3.0 9832 30.51261187957689
4009 68.0 38837 175.090763962201
4012 66.0 21108 312.677657760091
5035 90.0 47955 187.6759461995621
5095 17.0 6701 253.6934785852858
5099 24.0 8252 290.

In [305]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1852.0, 854757, 216.6697669630082)

In [333]:
total_state(Covid_death_capita) 

5952.198798368679 NEW MEXICO
15041.813586747394 ALABAMA
550.3376366998135 ALASKA
3924.592417085775 ARIZONA
15328.087839310409 ARKANSAS
4618.68397385468 CALIFORNIA
6814.056234159754 COLORADO
1410.8417882430933 CONNECTICUT
422.3456147033617 DELAWARE
10297.840384941668 FLORIDA
33420.91080383871 GEORGIA
4296.98620875796 IDAHO
20727.041615036644 ILLINOIS
17414.34236910441 INDIANA
21474.921115022342 IOWA
16024.683841690772 KANSAS
13582.97918574091 KENTUCKY
15108.68470721978 LOUISIANA
629.5480259662535 MAINE
2952.775001429612 MARYLAND
2544.2104175937834 MASSACHUSETTS
12207.908939508028 MICHIGAN
10769.179497309755 MINNESOTA
21454.879600493623 MISSISSIPPI
14595.493846644295 MISSOURI
9828.877583126487 MONTANA
11952.956910047997 NEBRASKA
1896.817057197103 NEVADA
662.4242989549832 NEW HAMPSHIRE
4868.717236246713 NEW JERSEY
7940.303311310678 NEW YORK
12232.106950714726 NORTH CAROLINA
12825.451311649205 NORTH DAKOTA
11049.338010245781 OHIO
8940.826231668423 OKLAHOMA
2033.8423816368042 OREGON
12525.6

{'NEW MEXICO': 5952.198798368679,
 'ALABAMA': 15041.813586747394,
 'ALASKA': 550.3376366998135,
 'ARIZONA': 3924.592417085775,
 'ARKANSAS': 15328.087839310409,
 'CALIFORNIA': 4618.68397385468,
 'COLORADO': 6814.056234159754,
 'CONNECTICUT': 1410.8417882430933,
 'DELAWARE': 422.3456147033617,
 'FLORIDA': 10297.840384941668,
 'GEORGIA': 33420.91080383871,
 'IDAHO': 4296.98620875796,
 'ILLINOIS': 20727.041615036644,
 'INDIANA': 17414.34236910441,
 'IOWA': 21474.921115022342,
 'KANSAS': 16024.683841690772,
 'KENTUCKY': 13582.97918574091,
 'LOUISIANA': 15108.68470721978,
 'MAINE': 629.5480259662535,
 'MARYLAND': 2952.775001429612,
 'MASSACHUSETTS': 2544.2104175937834,
 'MICHIGAN': 12207.908939508028,
 'MINNESOTA': 10769.179497309755,
 'MISSISSIPPI': 21454.879600493623,
 'MISSOURI': 14595.493846644295,
 'MONTANA': 9828.877583126487,
 'NEBRASKA': 11952.956910047997,
 'NEVADA': 1896.817057197103,
 'NEW HAMPSHIRE': 662.4242989549832,
 'NEW JERSEY': 4868.717236246713,
 'NEW YORK': 7940.303311310

In [306]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [334]:
Covid_death_capita[72049]

nan

In [307]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [308]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [309]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [310]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76277412.0 1000000 10122.288889402804
ALABAMA 957116.0000000001 76277412.0 1000000 12547.830018144821
ALASKA 231145.0 76277412.0 1000000 3030.3204309029256
ARIZONA 1839932.0 76277412.0 1000000 24121.583988717393
ARKANSAS 830467.0000000003 76277412.0 1000000 10887.456433367199
CALIFORNIA 11847711.0 76277412.0 1000000 155323.9771690209
COLORADO 1337805.0 76277412.0 1000000 17538.67842291241
CONNECTICUT 874974.0 76277412.0 1000000 11470.945028916292
DELAWARE 239009.0 76277412.0 1000000 3133.417793461582
FLORIDA 3892551.999999999 76277412.0 1000000 51031.516381284666
GEORGIA 1928702.999999999 76277412.0 1000000 25285.37543984842
IDAHO 340742.0000000001 76277412.0 1000000 4467.141596256571
ILLINOIS 2987496.0000000005 76277412.0 1000000 39166.195098491284
INDIANA 1602975.9999999995 76277412.0 1000000 21015.08110946396
IOWA 699741.0 76277412.0 1000000 9173.633211362756
KANSAS 401103.0000000001 76277412.0 1000000 5258.47678209114
KENTUCKY 1416013.0000000002 76277412.0 10000

In [311]:
print (Covid_state)

{'NEW MEXICO': 6527.0, 'ALABAMA': 22109.0, 'ALASKA': 2126.0, 'ARIZONA': 41939.0, 'ARKANSAS': 18464.0, 'CALIFORNIA': 168340.0, 'COLORADO': 16259.0, 'CONNECTICUT': 16400.0, 'DELAWARE': 4932.0, 'FLORIDA': 108180.0, 'GEORGIA': 55743.0, 'IDAHO': 4550.0, 'ILLINOIS': 35499.0, 'INDIANA': 22465.0, 'IOWA': 9684.0, 'KANSAS': 10400.0, 'KENTUCKY': 26762.0, 'LOUISIANA': 20303.0, 'MAINE': 3260.0, 'MARYLAND': 17239.0, 'MASSACHUSETTS': 31605.0, 'MICHIGAN': 19098.0, 'MINNESOTA': 11067.0, 'MISSISSIPPI': 12003.0, 'MISSOURI': 14147.0, 'MONTANA': 3630.0, 'NEBRASKA': 5545.0, 'NEVADA': 10007.0, 'NEW HAMPSHIRE': 4832.0, 'NEW JERSEY': 49210.0, 'NEW YORK': 96989.0, 'NORTH CAROLINA': 65737.0, 'NORTH DAKOTA': 1023.0, 'OHIO': 42929.0, 'OKLAHOMA': 25265.0, 'OREGON': 7354.0, 'PENNSYLVANIA': 50238.0, 'RHODE ISLAND': 7074.0, 'SOUTH CAROLINA': 40661.0, 'SOUTH DAKOTA': 1923.0, 'TENNESSEE': 31410.0, 'TEXAS': 196151.0, 'UTAH': 14456.0, 'VERMONT': 1624.0, 'VIRGINIA': 43197.0, 'WASHINGTON': 15451.0, 'WEST VIRGINIA': 7015.0, 

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [312]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [329]:
for j in Covid_death_capita:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])
    print(Proportional_to_covid_death_cap[j])


35039 61.0 38921 156.72773053107576


KeyError: 35039

In [330]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [328]:
Total_covid_death_cap


{'NEW MEXICO': 5952.198798368679,
 'ALABAMA': 15041.813586747394,
 'ALASKA': 550.3376366998135,
 'ARIZONA': 3924.592417085775,
 'ARKANSAS': 15328.087839310409,
 'CALIFORNIA': 4618.68397385468,
 'COLORADO': 6814.056234159754,
 'CONNECTICUT': 1410.8417882430933,
 'DELAWARE': 422.3456147033617,
 'FLORIDA': 10297.840384941668,
 'GEORGIA': 33420.91080383871,
 'IDAHO': 4296.98620875796,
 'ILLINOIS': 20727.041615036644,
 'INDIANA': 17414.34236910441,
 'IOWA': 21474.921115022342,
 'KANSAS': 16024.683841690772,
 'KENTUCKY': 13582.97918574091,
 'LOUISIANA': 15108.68470721978,
 'MAINE': 629.5480259662535,
 'MARYLAND': 2952.775001429612,
 'MASSACHUSETTS': 2544.2104175937834,
 'MICHIGAN': 12207.908939508028,
 'MINNESOTA': 10769.179497309755,
 'MISSISSIPPI': 21454.879600493623,
 'MISSOURI': 14595.493846644295,
 'MONTANA': 9828.877583126487,
 'NEBRASKA': 11952.956910047997,
 'NEVADA': 1896.817057197103,
 'NEW HAMPSHIRE': 662.4242989549832,
 'NEW JERSEY': 4868.717236246713,
 'NEW YORK': 7940.303311310

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [315]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [316]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [317]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [318]:
#print (Medicaid_demand)

In [319]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [320]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [321]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [322]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)